In [1]:
# Import packages
import pandas as pd, numpy as np
import os, sys, glob, re
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
inputfolder = Path(r'C:\Users\jasonjia\Dropbox\projects\combine_dafne_data\data\Dafne CSV Raw Data')
outputfolder_fragments = Path(r'C:\Users\jasonjia\Dropbox\projects\combine_dafne_data\output\fragments')
outputfolder_combined = Path(r'C:\Users\jasonjia\Dropbox\projects\combine_dafne_data\output\combined')
colnamesfilepath = Path(r'C:\Users\jasonjia\Dropbox\projects\combine_dafne_data\data\examples\Dafne_Check_2.xlsx')

In [3]:
# Get list of files to combine
inputfilename_list = os.listdir(inputfolder)
print("Number of files:", len(inputfilename_list))

Number of files: 678


# Define column names

In [4]:
def process_colnames(colnames, replacements):
    for replacement in replacements:
        colnames = colnames.str.replace(replacement[0], replacement[1], regex=False)
    return colnames

In [5]:
# Get column names
df_colnames = pd.read_excel(colnamesfilepath, sheet_name=1)
colnames = df_colnames.columns
colnames = colnames.str.replace('\n',' ')
colnames = colnames.str.lower()
colnames = colnames[1:] # Drop the first column, which is just an index column

# Process column names to comply with Stata requirements:
# strings only, max 32 characters, only alphanumerics and underscores, no Stata reserved words
replacements = [[',', ''], [' ', '_'] , ['/_', ''], ['(tc)_', ''], ['th_eur_', ''], ['.1', '_1'], ['/', '_'], ['-_', ''],
                ['up_to_1_year', '0_1_yr'], ['1_to_5_years', '1_5_yr'], ['more_than_1_and_up_to_5_years', '1_5_yr'], 
                ['thereof', 'thof'], ['primary_activity_code', 'pri_actv_cd'],
                ['above_5_years', '5_plus_yr'], ['more_than_5_years', '5_plus_yr'], 
                ['more_than_1_year', '1_plus_yr'], ['for_the_year_', ''], ['from', 'fr'], 
                ['(corporates)_', ''], ['shareholder', 'shhldr'], ['share', 'sh'], ['capital', 'cap'], ['account', 'acct'],
                ['interest', 'int'], ['ordinary', 'ord'], ['activities', 'actvs'], ['subscribed', 'subd'], ['liabilities', 'liabs'],
                ['cost_of_raw_materials_consumables_and_supplies_and_of_purchased_merchandise', 'cost_of_raw_materials'],
                ['increase_or_decrease_in', 'inc_or_dec'], ['stocks_of_animals', 'stocks_animals'],
                ['finished_goods_inventories_and_work_in_process', 'fin_gds_inv'],
                ['bank_deposits_and_fr_receivables_to_third_parties', 'bank_deposits']]
colnames = process_colnames(colnames, replacements)

# View processed column names
for i in colnames:
    print(i)

company_name
crefo_number
gkz_administrative_region_code
gkz_borough_code
wz_2008_pri_actv_cd_1_digit
wz_2008_pri_actv_cd_3_digits
original_date_of_incorporation
listed_delisted_unlisted
peer_group_name
peer_group_description
number_of_employees_2014
number_of_employees_2013
number_of_employees_2012
number_of_employees_2011
number_of_employees_2010
number_of_employees_2009
number_of_employees_2008
number_of_employees_2007
number_of_employees_2006
number_of_employees_2005
tangible_assets_2014
tangible_assets_2013
tangible_assets_2012
tangible_assets_2011
tangible_assets_2010
tangible_assets_2009
tangible_assets_2008
tangible_assets_2007
tangible_assets_2006
tangible_assets_2005
on_tangible_assets_2014
on_tangible_assets_2013
on_tangible_assets_2012
on_tangible_assets_2011
on_tangible_assets_2010
on_tangible_assets_2009
on_tangible_assets_2008
on_tangible_assets_2007
on_tangible_assets_2006
on_tangible_assets_2005
gross_results_2014
gross_results_2013
gross_results_2012
gross_results_201

# Combine files into 5 .csv fragments (additional.csv + 4 .csv fragments for everything else)

In [6]:
def combine_files(inputfolder, inputfilename_list, outputfilepath, colnames):
    df_combined = pd.DataFrame()
    for i, inputfilename in enumerate(inputfilename_list):
        print(f"Combining {i}: {inputfilename}")
        inputfilepath = Path(inputfolder / inputfilename)
        # The standard utf-8 encoding does not work, but cp1252 encoding does.
        df = pd.read_csv(inputfilepath, encoding='cp1252', low_memory=False, header=None, skip_blank_lines=True)
        # Drop the first column, which is just an index column
        df = df.iloc[:,1:]
        df.columns = colnames
        df['source_filename'] = inputfilename
        # Note that there seems to be many blank lines. But actually they have one column (306) with an int value. 
        # This is the column for US SIC codes, where a firm can have multiple SIC codes.
        # Fill in the missing values using the closest row above with non-missing values (forward-fill).
        df = df.fillna(method='ffill')
        df = df.replace('n.a.', np.NaN)
        df_combined = pd.concat([df_combined, df])
    if outputfilepath != '':
        df_combined.to_csv(outputfilepath, index=False)
        print("Saved file as:", outputfilepath)
        print("\n---\n")
    return df_combined

In [7]:
# Combine files into .csv fragments
# Treat additional.csv separately
# combine_files(inputfolder, inputfilename_list[1:2], Path(outputfolder_fragments / 'test.csv'), colnames)
combine_files(inputfolder, inputfilename_list[0:1], Path(outputfolder_fragments / 'additional.csv'), colnames)
combine_files(inputfolder, inputfilename_list[1:150], Path(outputfolder_fragments / 'dafne_export_fragment_1.csv'), colnames)
combine_files(inputfolder, inputfilename_list[150:300], Path(outputfolder_fragments / 'dafne_export_fragment_2.csv'), colnames)
combine_files(inputfolder, inputfilename_list[300:450], Path(outputfolder_fragments / 'dafne_export_fragment_3.csv'), colnames)
combine_files(inputfolder, inputfilename_list[450:], Path(outputfolder_fragments / 'dafne_export_fragment_4.csv'), colnames)

Combining 0: additional.csv
Saved file as: C:\Users\jasonjia\Dropbox\projects\combine_dafne_data\output\fragments\additional.csv

---

Combining 0: Dafne_Export_1.xlsx.csv
Combining 1: Dafne_Export_10.xlsx.csv
Combining 2: Dafne_Export_11.xlsx.csv
Combining 3: Dafne_Export_12.xlsx.csv
Combining 4: Dafne_Export_13.xlsx.csv
Combining 5: Dafne_Export_14.xlsx.csv
Combining 6: Dafne_Export_15.xlsx.csv
Combining 7: Dafne_Export_16.xlsx.csv
Combining 8: Dafne_Export_17.xlsx.csv
Combining 9: Dafne_Export_18.xlsx.csv
Combining 10: Dafne_Export_19.xlsx.csv
Combining 11: Dafne_Export_1_34 (1).xlsx.csv
Combining 12: Dafne_Export_1_34 (10).xlsx.csv
Combining 13: Dafne_Export_1_34 (11).xlsx.csv
Combining 14: Dafne_Export_1_34 (12).xlsx.csv
Combining 15: Dafne_Export_1_34 (13).xlsx.csv
Combining 16: Dafne_Export_1_34 (14).xlsx.csv
Combining 17: Dafne_Export_1_34 (15).xlsx.csv
Combining 18: Dafne_Export_1_34 (16).xlsx.csv
Combining 19: Dafne_Export_1_34 (17).xlsx.csv
Combining 20: Dafne_Export_1_34 (1

,company_name,crefo_number,gkz_administrative_region_code,gkz_borough_code,wz_2008_pri_actv_cd_1_digit,wz_2008_pri_actv_cd_3_digits,original_date_of_incorporation,listed_delisted_unlisted,peer_group_name,peer_group_description,...,thof_int_fr_bank_deposits_2013,thof_int_fr_bank_deposits_2012,thof_int_fr_bank_deposits_2011,thof_int_fr_bank_deposits_2010,thof_int_fr_bank_deposits_2009,thof_int_fr_bank_deposits_2008,thof_int_fr_bank_deposits_2007,thof_int_fr_bank_deposits_2006,thof_int_fr_bank_deposits_2005,source_filename
0,Mocca-X-Press GmbH & Co. KG,2.011802e+09,11.0,11000.0,I,I56,11/09/2006,Nicht börsennotiert,I561 - SM,"Restaurants, Gaststätten, Imbissstuben, Cafés,...",...,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,Dafne_Export_35_513 (459).xlsx.csv
1,MOCC Mobile Office Competence Center Be- teili...,8.250344e+09,9.0,9171.0,M,M70,2006,Nicht börsennotiert,M701 - SM,Verwaltung und Führung von Unternehmen und Bet...,...,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,Dafne_Export_35_513 (459).xlsx.csv
2,MOCC Mobile Office Competence Center Be- teili...,8.250344e+09,9.0,9171.0,M,M70,2006,Nicht börsennotiert,M701 - SM,Verwaltung und Führung von Unternehmen und Bet...,...,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,Dafne_Export_35_513 (459).xlsx.csv
3,MOC-Consult GmbH,7.330599e+09,8.0,8111.0,M,M70,06/07/2000,Nicht börsennotiert,M702 - LA,Public-Relations- und Unternehmensberatung (GR...,...,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,Dafne_Export_35_513 (459).xlsx.csv
4,MOC-Consult GmbH,7.330599e+09,8.0,8111.0,M,M70,06/07/2000,Nicht börsennotiert,M702 - LA,Public-Relations- und Unternehmensberatung (GR...,...,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,n.v.,Dafne_Export_35_513 (459).xlsx.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6158,Kunkel Consulting International GmbH,6.270118e+09,6.0,6431.0,M,M71,08/10/2003,Unlisted,M711 - LA,Architectural and engineering activities and r...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dafne_Export_9.xlsx.csv
6159,Kunkel Consulting International GmbH,6.270118e+09,6.0,6431.0,M,M71,08/10/2003,Unlisted,M711 - LA,Architectural and engineering activities and r...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dafne_Export_9.xlsx.csv
6160,Langlotz Glas- und Gebäudereinigung GmbH,7.210131e+09,8.0,8317.0,N,N81,08/03/2011,Unlisted,N812 - LA,Cleaning activities (large companies),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dafne_Export_9.xlsx.csv
6161,Lumitronic GmbH,5.230279e+09,5.0,5166.0,F,F43,01/10/1996,Unlisted,F432 - LA,"Electrical, plumbing and other construction in...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dafne_Export_9.xlsx.csv


# Combine fragments into 1 combined file, and save as both .csv and .dta

In [8]:
def combine_fragments(inputfolder, inputfilename_list, outputfolder, outputfilename):
    df_combined = pd.DataFrame()
    for i, inputfilename in enumerate(inputfilename_list):
        print(f"Combining {i}: {inputfilename}")
        inputfilepath = Path(inputfolder / inputfilename)
        df = pd.read_csv(inputfilepath, low_memory=False)
        df_combined = pd.concat([df_combined, df])
    if outputfolder != '':
        outputfilepath = Path(outputfolder / str(outputfilename+'.csv'))
        df_combined.to_csv(outputfilepath, index=False)
        print("Saved file as:", outputfilepath)
        outputfilepath = Path(outputfolder / str(outputfilename+'.dta'))
        df_combined.to_stata(outputfilepath, write_index=False)
        print("Saved file as:", outputfilepath)
        print("\n---\n")
    return df_combined

In [9]:
# Get list of fragments to combine
fragmentfilename_list = os.listdir(outputfolder_fragments)
print("Number of files:", len(fragmentfilename_list))
fragmentfilename_list

Number of files: 5


['additional.csv',
 'dafne_export_fragment_1.csv',
 'dafne_export_fragment_2.csv',
 'dafne_export_fragment_3.csv',
 'dafne_export_fragment_4.csv']

In [ ]:
# Combine fragments into .csv and .dta files
# combine_fragments(outputfolder_fragments, fragmentfilename_list, outputfolder_combined, 'test')
combine_fragments(outputfolder_fragments, fragmentfilename_list, outputfolder_combined, 'dafne_export_combined_20230217')

Combining 0: additional.csv
Combining 1: dafne_export_fragment_1.csv
Combining 2: dafne_export_fragment_2.csv
Combining 3: dafne_export_fragment_3.csv
Combining 4: dafne_export_fragment_4.csv
Saved file as: C:\Users\jasonjia\Dropbox\projects\combine_dafne_data\output\combined\dafne_export_combined_20230217.csv
